In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/disaster-tweets/tweets.csv')

In [ ]:
df.head()

We would take only useful columns here that is text and target

In [ ]:
df_1 = df[['text','target']]

In [ ]:
df_1.head()

In [ ]:
y = df_1['target']
y = np.array(y)
y

In [ ]:
x = df_1.text.values

In [ ]:
x

our target and feature columns have been made now some text pre processing

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x)

#tokenizer.word_index


to check whcih word has been assigned what value

In [ ]:
len(tokenizer.word_index)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(x)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

Visualize text preprocessing

In [ ]:
x[0]

In [ ]:
encoded_docs[0]

In [ ]:
padded_sequence[0]

In [ ]:
padded_sequence.shape

In [ ]:
#Building LSTM MODEL
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,     
                                     input_length=200) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(120,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(40))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(padded_sequence,y,
                  validation_split=0.2, epochs=7, batch_size=32)


In [ ]:
def predict(tweet):
    test_word = tweet
    tw = tokenizer.texts_to_sequences([test_word])
    tw = pad_sequences(tw,maxlen=200)
    prediction = int(model.predict(tw).round().item())
    if(prediction==0):
        return('No Disaster')
    else:
        return('Disaster')


In [ ]:
predict('Breaking news:Nigeria flag set ablaze in Aba.')

In [ ]:
predict('On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE')

In [ ]:
predict("Evacuation drill at work. The fire doors wouldn't open so i got to smash the emergency release glass #feelingmanly")